# Importation des librairies et datas

In [ ]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Masking, Embedding, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn import metrics
from sklearn import linear_model
from keras.models import Sequential
import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import requests
import json

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [36]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [56]:
data = pd.read_csv('gdrive/MyDrive/Nlp_projet/data_nlp.csv',encoding='ISO-8859-1',header=None,names=['feeling','ids','date','query','user','text'])
data.drop(['ids','date','query','user'], axis=1, inplace=True) #on garde les colonnes qui nous intéressent
data_size = 10000

#sélection de 20000 échantillons pour accélerer les tests
negative_data=data['text'][:data_size]
positive_data=data['text'][800000:800000+data_size]

negative_feelings=data['feeling'][:data_size]
positive_feelings=data['feeling'][800000:800000+data_size]

# Preprocessing

In [57]:
#Remplissage de la liste indices sentiments
def analyze(data):
  feelings_col=[]
  for line in data:
    feelings_col.append(line)

  return feelings_col

positive_feelings_data = analyze(positive_feelings)
negative_feelings_data = analyze(negative_feelings)
feelings_data = positive_feelings_data + negative_feelings_data


#Régulation de la ponctuation 
def punct_remover(data):
  removed_punct = string.punctuation
  punct=[]
  for i in range(len(removed_punct)):
    punct.append(removed_punct[i])


  no_punct_dataset=[]
  element=""
  for elt in data:
    for letter in elt:
      if letter not in punct and not letter.isdigit():
        element += letter

    no_punct_dataset.append(element)
    element =""
  
  return no_punct_dataset

Analyzed_positive_data=punct_remover(positive_data)
Analyzed_negative_data=punct_remover(negative_data)
Analyzed_data=Analyzed_positive_data + Analyzed_negative_data

#Création d'un dataframe texte+indice sentiment
frame = pd.DataFrame(Analyzed_data, columns = ["text"])
frame["feelings"] = feelings_data
frame

,text,feelings
0,I LOVE HealthUandPets u guys r the best,4
1,im meeting up with one of my besties tonight C...,4
2,DaRealSunisaKim Thanks for the Twitter add Sun...,4
3,Being sick can be really cheap when it hurts t...,4
4,LovesBrooklyn he has that effect on everyone,4
...,...,...
19995,Aww thats sad,0
19996,stupid dvds stuffing up the good bits in jaws,0
19997,DandySephy No Only close friends and family Im...,0
19998,CRAP After looking when I last tweeted WHY AM ...,0


In [58]:
#Mise en minuscule des textes
def data_lower(data):
  for i in range(len(data["text"])):
    data["text"][i]=data["text"][i].lower()
  
  return data

lower_frame = data_lower(frame)
lower_frame

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,text,feelings
0,i love healthuandpets u guys r the best,4
1,im meeting up with one of my besties tonight c...,4
2,darealsunisakim thanks for the twitter add sun...,4
3,being sick can be really cheap when it hurts t...,4
4,lovesbrooklyn he has that effect on everyone,4
...,...,...
19995,aww thats sad,0
19996,stupid dvds stuffing up the good bits in jaws,0
19997,dandysephy no only close friends and family im...,0
19998,crap after looking when i last tweeted why am ...,0


In [75]:
#Le texte est transformé en séquence d'entiers puis un padding est appliqué
tokenizer = Tokenizer(num_words=data_size, split=" ")
tokenizer.fit_on_texts(lower_frame["text"].values)
Vectorized_text = tokenizer.texts_to_sequences(lower_frame["text"].values)

Vectorized_text = pad_sequences(X)
print(Vectorized_text[:33])
print(Vectorized_text.shape)

[[   0    0    0 ...  388    3  179]
 [   0    0    0 ...  142  294  324]
 [   0    0    0 ...  131    4 2092]
 ...
 [   0    0    0 ...   25    3  422]
 [   0    0    0 ...  712   18  109]
 [   0    0    0 ... 9500 9501 9502]]
(20000, 32)


In [80]:
#Retourne des tuples positifs/négatifs ([1 0] négatif et [0 1] positif)
tuple_feelings = pd.get_dummies(frame["feelings"]).values
print(tuple_feelings)
print(tuple_feelings.shape)

[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]
(20000, 2)


# Recurrent Neural Network

In [63]:
model = Sequential()

#Transforme les entiers positifs (index) en vecteurs de tailles fixes
#Une régulation sur le réseau est appliquée
model.add(Embedding(data_size, 256, input_length=X.shape[1]))
model.add(Dropout(0.3))

#Utilisation d'un réseau de longue mémoire à courte terme pour allonger la mémoire
model.add(LSTM(256,return_sequences=False,dropout=0.3,recurrent_dropout=0.2))
model.add(Dense(2, activation="sigmoid"))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

#On sépare la data entre l'entrainement et la validation du modèle 
x_train,x_test,y_train,y_test = train_test_split(X,Y,train_size=0.7,random_state=0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 256)           5120000   
_________________________________________________________________
dropout (Dropout)            (None, 32, 256)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 5,645,826
Trainable params: 5,645,826
Non-trainable params: 0
_________________________________________________________________


# Entrainement et sauvegarde du modèle

In [64]:
model.fit(x_train,  y_train, 
                   epochs=8, batch_size=100,verbose =2)

Epoch 1/8
140/140 - 94s - loss: 0.5911 - accuracy: 0.6738
Epoch 2/8
140/140 - 90s - loss: 0.4422 - accuracy: 0.8012
Epoch 3/8
140/140 - 91s - loss: 0.3600 - accuracy: 0.8417
Epoch 4/8
140/140 - 91s - loss: 0.2860 - accuracy: 0.8817
Epoch 5/8
140/140 - 91s - loss: 0.2294 - accuracy: 0.9073
Epoch 6/8
140/140 - 92s - loss: 0.1847 - accuracy: 0.9250
Epoch 7/8
140/140 - 102s - loss: 0.1510 - accuracy: 0.9395
Epoch 8/8
140/140 - 91s - loss: 0.1284 - accuracy: 0.9509


In [65]:
model.evaluate(x_test,y_test)

188/188 [==============================] - 9s 48ms/step - loss: 0.9639 - accuracy: 0.7195


[0.9639269113540649, 0.7195000052452087]

In [67]:
model.save('/content/gdrive/MyDrive/Nlp_projet/Model1')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Nlp_projet/Model1/assets


# Récupération d'informations de L' API Yelp

In [100]:
api_key='q1HJoEmRMdypie_E6lYwiih0hNfuSatSTQXH6Bh9fnltm30JX4XO1TSCd6NARRVszrs4lMXb7N3i6LWB_UIBEinyG_ah-0expG6GUnzTdUss2gvNQRcJ7fJhK5g2YHYx'
headers = {'Authorization': 'Bearer %s' % api_key}


url = "https://api.yelp.com/v3/businesses/FEVQpbOPOwAPNIgO7D3xxw/reviews"
req = requests.get(url, headers=headers)
print('the status code is {}'.format(req.status_code))


data=json.loads(req.text)
data

the status code is 200


{'possible_languages': ['fr', 'en', 'zh', 'pt', 'de', 'it', 'sv', 'ja', 'es'],
 'reviews': [{'id': 'qOiuCTz_Gzgmose1GHaAlg',
   'rating': 5,
   'text': "Truth be told if it was up to me I'd be giving 4/5 stars, we did order recently and our cheese fries weren't delivered, couldn't speak with anyone from...",
   'time_created': '2020-12-19 07:52:35',
   'url': 'https://www.yelp.com/biz/shake-shack-new-york-2?adjust_creative=KzZVmcfbYqCMY9omoha5NA&hrid=qOiuCTz_Gzgmose1GHaAlg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=KzZVmcfbYqCMY9omoha5NA',
   'user': {'id': 'FRKnLBnlFiasr1Dc8oOIGQ',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/1wm0yjWSw92j_ZsUFZBGzQ/o.jpg',
    'name': 'Sarah G.',
    'profile_url': 'https://www.yelp.com/user_details?userid=FRKnLBnlFiasr1Dc8oOIGQ'}},
  {'id': '7d56A_ObMPHHyywcfhnrUw',
   'rating': 5,
   'text': 'Happened to be in the city today and not too far from here so I had to stop by and pick up cheese fries (I love those c

# Traitement des informations

In [101]:
#Application du preprocessing sur les informations importées
def traitement(tweet):
    
  user_text=punct_remover(tweet)

  frame_string=pd.DataFrame(user_text, columns = ["text"])
  test_lower=data_lower(frame_string)

  tokenizer = Tokenizer(num_words=data_size, split=" ")
  tokenizer.fit_on_texts(test_lower["text"].values)

  pp_string = tokenizer.texts_to_sequences(test_lower["text"].values)
  pp_string = pad_sequences(pp_string)

  return pp_string

# Utilisation du modèle

In [102]:
text1 =data["reviews"][0]["text"]
text2="I went to this cupcakes shop after the recommendations I got from the comments and I am utterly disappointed. Whether they made their comments themselves or people have a very bad sense of taste but I do not recommend any of you to go there."
text3="Very good restaurant with a lovely staff."  

test1 = traitement(text1)
test2 = traitement(text2)
test3 = traitement(text3)

preds1 = model.predict(test1)
preds2 = model.predict(test2)
preds3 = model.predict(test3)
prediction_list=[preds1,preds2,preds3]
target_feeling=["positive","negative","positive"]

for i in range(len(prediction_list)):
  if prediction_list[i][0][0] < (1/3):
    print('predicted feeling : negative')
    print(f"target feeling : {target_feeling[i]}")
  elif  prediction_list[i][0][0] > (2/3): 
    print('predicted feeling : positive')
    print(f"target feeling : {target_feeling[i]}")
  else:
    print('predicted feeling : neutral')
    print(f"target feeling : {target_feeling[i]}")

predicted feeling : positive
target feeling : positive
predicted feeling : positive
target feeling : negative
predicted feeling : neutral
target feeling : positive
